In [47]:
import numpy as np
import sympy
import re
from fractions import Fraction

Core Testing

>Solution Function

>>Solution Function 1

In [9]:
def solve_linear_equations(coefficients, constants):
    try:
        # Trying to find a unique solution
        solution = np.linalg.solve(coefficients, constants)
    except np.linalg.LinAlgError:
        # Handle overdetermined systems
        solution, _, _, _ = np.linalg.lstsq(coefficients, constants, rcond=None)
    
    return solution


>>Solution Function 2

In [17]:
def solve_linear_equations(coefficients, constants):
    try:
        # Trying to find a unique solution
        solution = np.linalg.solve(coefficients, constants)
        return solution, "Unique solution"
    except np.linalg.LinAlgError as e:
        if 'Singular matrix' in str(e):
            # This means the matrix is singular and there's no unique solution
            # This can be due to no solutions or infinite solutions
            # We'll use lstsq to determine which one it is
            solution, residuals, rank, s = np.linalg.lstsq(coefficients, constants, rcond=None)
            if residuals.size == 0 or np.allclose(residuals, 0):
                return solution, "Infinite solutions"
            else:
                return solution, "No solution"
        else:
            # This is some other kind of linear algebra error we didn't expect
            raise

>>>Solution Function 3 (Handles Non-square Matrix)

In [23]:
def solve_linear_equations(coefficients, constants):
    # If matrix is not square, use lstsq to find the least squares solution
    if coefficients.shape[0] != coefficients.shape[1]:
        solution, residuals, rank, s = np.linalg.lstsq(coefficients, constants, rcond=None)
        if residuals.size == 0 or np.allclose(residuals, 0):
            return solution, "Infinite solutions"
        else:
            return solution, "No solution"
    else:
        try:
            solution = np.linalg.solve(coefficients, constants)
            return solution, "Unique solution"
        except np.linalg.LinAlgError as e:
            if 'Singular matrix' in str(e):
                return None, "No solution"
            else:
                raise

>>>Solution Function 4 (Handles Parametrics)

In [49]:
def solve_linear_equations(coefficients, constants):
    # Perform Gaussian elimination or row reduction to obtain RREF
    rref_matrix = np.hstack((coefficients, constants.reshape(-1, 1)))
    rref_matrix = sympy.Matrix(rref_matrix).rref()[0]
    rref_matrix = np.array(rref_matrix.tolist(), dtype=float)

    num_rows, num_cols = rref_matrix.shape
    free_vars = num_cols - num_rows - 1  # -1 for the constants column

    if free_vars > 0:
        # Infinite solutions, proceed to express in terms of free variables
        solution_set = []
        for i in range(num_rows):
            leading_coeff = rref_matrix[i][i]
            expression = f"{rref_matrix[i, -1]/leading_coeff}"
            for j in range(i + 1, num_cols - 1):
                if rref_matrix[i, j] != 0:
                    expression += f" - {rref_matrix[i, j]/leading_coeff}*x{j+1}"
            solution_set.append(expression)
        # Add the free variables
        for i in range(free_vars):
            solution_set.append(f"x{num_rows+i+1}")
        
        parametric_solution = ', '.join(solution_set)
        return parametric_solution, "Infinite solutions"
    else:
        # No free variables, system has a unique solution
        solution = np.linalg.solve(coefficients, constants)
        return solution, "Unique solution"


>>>Solution Function 5 (Handles Inconsistent/No Solution) **WORKING - IN USE**

In [74]:
def solve_linear_equations(coefficients, constants):
    # Augment the coefficients matrix with the constants vector
    augmented_matrix = np.hstack((coefficients, constants.reshape(-1, 1)))
    # Convert to a sympy Matrix and compute the RREF
    rref_matrix, pivot_columns = sympy.Matrix(augmented_matrix).rref()
    rref_matrix = np.array(rref_matrix.tolist(), dtype=float)

    # Check for inconsistency: a row of zeros in coefficients with a non-zero constant
    for i in range(len(rref_matrix)):
        if np.allclose(rref_matrix[i, :-1], 0) and not np.isclose(rref_matrix[i, -1], 0):
            return None, "No solution"

    num_rows, num_cols = rref_matrix.shape
    free_vars = num_cols - num_rows - 1  # -1 for the constants column

    if free_vars > 0:
        # Infinite solutions, proceed to express in terms of free variables
        solution_set = []
        for i in range(num_rows):
            if i in pivot_columns:
                leading_coeff = rref_matrix[i][i]
                expression = f"{rref_matrix[i, -1]/leading_coeff}"
                for j in range(i + 1, num_cols - 1):
                    if rref_matrix[i, j] != 0:
                        expression += f" - {rref_matrix[i, j]/leading_coeff}*x{j+1}"
                solution_set.append(expression)
        # Add expressions for free variables
        for i in range(num_rows, num_cols - 1):  # -1 to exclude the constants column
            if i not in pivot_columns:
                solution_set.append(f"x{i+1} is a free variable")
        
        parametric_solution = ', '.join(solution_set)
        return parametric_solution, "Infinite solutions"
    elif len(pivot_columns) == num_rows:
        # Unique solution, extract it from the last column of the RREF
        unique_solution = rref_matrix[:, -1]
        return unique_solution, "Unique solution"
    else:
        # Underdetermined system with no unique solution
        return None, "Infinite solutions, cannot express as unique solution"


System Size Checking Function

In [ ]:
def check_system_size(num_equations, num_variables):
    if num_variables > 5 or num_equations > 5:
        raise ValueError("This system can only handle up to a 5x5 matrix.")
    if num_variables < 1 or num_equations < 1:
        raise ValueError("This system requires at least a 1x1 matrix.")


Solution Formating Function

In [41]:
def format_parametric_solution(solution_str):
    # Split the solution string into individual variable components
    variables = solution_str.split(',')

    # Define a pattern to match the variables like 'x4' and 'x5'
    var_pattern = re.compile(r'x\d+')

    formatted_solution = []

    for i, var in enumerate(variables):
        # Find all free variables in the expression
        free_vars = var_pattern.findall(var)

        # Correctly identify negative and positive floats
        expr = re.sub(r'(?<!\w)-?\d+\.\d+', lambda m: f"{float(m.group(0)):0.3f}", var)

        # If the variable part is a free variable, format it accordingly
        if i + 1 in [int(free_var[1:]) for free_var in free_vars]:
            formatted_solution.append(f"x{i+1} is a free variable")
        else:
            # Remove unnecessary pluses and correct double negatives
            expr = expr.replace(' - -', ' + ').replace('+-', '-')
            formatted_solution.append(f"x{i+1} = {expr.strip()}")

    return formatted_solution

>Solution Formating 2 (Parametric w/ fractions) **WORKING - IN USE**

In [80]:
def format_parametric_solution(solution_str):
    # Split the solution string into individual variable components
    variables = solution_str.split(',')

    # Define a pattern to match the variables like 'x4' and 'x5'
    var_pattern = re.compile(r'x\d+')

    formatted_solution = []

    for i, var in enumerate(variables):
        # Find all free variables in the expression
        free_vars = var_pattern.findall(var)

        # Correctly identify negative and positive floats
        expr = re.sub(r'(?<!\w)-?\d+\.\d+', lambda m: str(Fraction(float(m.group(0))).limit_denominator()), var)

        # If the variable part is a free variable, format it accordingly
        if i + 1 in [int(free_var[1:]) for free_var in free_vars]:
            formatted_solution.append(f"x{i+1} is a free variable")
        else:
            # Remove unnecessary pluses and correct double negatives
            expr = expr.replace(' - -', ' + ').replace('+-', '-')
            formatted_solution.append(f"x{i+1} = {expr.strip()}")

    return formatted_solution

Input Segment

In [24]:
num_variables = int(input("Enter the number of variables (up to 5): "))
num_equations = int(input("Enter the number of equations: "))

In [25]:
coefficients = []
constants = []

for i in range(num_equations):
    equation = input(f"Enter coefficients and constant for equation {i+1}, separated by space: ")
    values = [float(x) for x in equation.split()]
    coefficients.append(values[:-1])  # All but last
    constants.append(values[-1])      # Last element

In [19]:
coefficients = []
constants = []

for i in range(num_equations):
    # Ask the user to enter only the coefficients and constants as numbers separated by spaces
    equation = input(f"Enter the coefficients for equation {i+1} followed by the constant, separated by space: ")
    
    # Split the input string by spaces, convert each to a float, and append to the lists
    values = [float(x) for x in equation.split()]
    
    # Assuming the last number is the constant, and the rest are coefficients
    coefficients.append(values[:-1])  # All but last
    constants.append(values[-1])      # Last element


In [26]:
coefficients_array = np.array(coefficients)
constants_array = np.array(constants)

In [27]:
solution = solve_linear_equations(coefficients_array, constants_array)
print("Solution:", solution)

Solution: (array([ 0.16975854,  0.91372203, -0.21481154,  0.41740283,  0.14355124]), 'Infinite solutions')


Test

2x + 3y - z + w - 2v = 4

4x - y + 5z - 2w + v = -2

-x + 2y + 3z + w - 3v = 1

In [32]:
# Test:
coefficients_array = np.array([
    [2, 3, -1, 1, -2],
    [4, -1, 5, -2, 1],
    [-1, 2, 3, 1, -3]
])

constants_array = np.array([4, -2, 1])

In [35]:
solution, status = solve_linear_equations(coefficients_array, constants_array)
print("Status:", status)

Status: Infinite solutions


In [36]:
print("Solution:", solution)

Solution: 0.19047619047619047 - -0.27380952380952384*x4 - 0.32142857142857145*x5, 1.0952380952380953 - 0.4880952380952381*x4 - -0.9642857142857143*x5, -0.3333333333333333 - -0.08333333333333333*x4 - -0.25*x5, x4, x5


In [46]:
# Format the parametric solution
formatted_solution = format_parametric_solution(str(solution))

# Print the formatted solution
for line in formatted_solution:
    print(line)


x1 = 0.190 + 0.274*x4 - 0.321*x5
x2 = 1.095 - 0.488*x4 + 0.964*x5
x3 = -0.333 + 0.083*x4 + 0.250*x5
x4 is a free variable
x5 is a free variable


In [51]:
# Format the parametric solution
formatted_solution = format_parametric_solution(str(solution))

# Print the formatted solution
for line in formatted_solution:
    print(line)


x1 = 4/21 + 23/84*x4 - 9/28*x5
x2 = 23/21 - 41/84*x4 + 27/28*x5
x3 = -1/3 + 1/12*x4 + 1/4*x5
x4 is a free variable
x5 is a free variable


***TESTS:***

Unique Solution (5x5 system)

x + 2y + 3z - w + v = 1

2x - y + z + w - 3v = -1

3x + y - z - 2w + 2v = 3

-x + 3y + 2z + w + v = 4

x - y - z - w + v = 0

In [57]:
# Test:
coefficients_array = np.array([
    [1, 2, 3, -1, 2],
    [2, -1, 1, 1, -3],
    [3, 1, -1, -2, 2],
    [-1, 3, 2, 1, 1],
    [1, -1, -1, -1, 1]
])

constants_array = np.array([1, -1, 3, 4, 0])
solution, status = solve_linear_equations(coefficients_array, constants_array)
print("Status:", status)
print("Solution:", solution)
# Format the parametric solution
formatted_solution = format_parametric_solution(str(solution))

# Print the formatted solution
for line in formatted_solution:
    print(line)


Status: Unique solution
Solution: [ 1.25454545  0.83636364 -0.76363636  2.72727273  1.54545455]
x1 = [ 69/55  46/55 -42/55  30/11  17/11]


Infinite Solutions (3 equations, 5 variables) 

2x + 3y - z + w - 2v = 4

4x - y + 5z - 2w + v = -2

4x + 3y - 2z + 2w - 2v = 10

In [75]:
# Test:
coefficients_array = np.array([
    [2, 3, -1, 1, -2],
    [4, -1, 5, -2, 1],
    [4, 3, -2, 2, -2]
])

constants_array = np.array([4, -2, 10])

solution, status = solve_linear_equations(coefficients_array, constants_array)
print("Status:", status)
print("Solution:", solution)
# Format the parametric solution
formatted_solution = format_parametric_solution(str(solution))

# Print the formatted solution
for line in formatted_solution:
    print(line)


Status: Infinite solutions
Solution: 1.9523809523809523 - 0.21428571428571427*x4 - 0.023809523809523808*x5, -0.6666666666666666 - -0.6666666666666666*x5, -2.0952380952380953 - -0.5714285714285714*x4 - 0.047619047619047616*x5, x4 is a free variable, x5 is a free variable
x1 = 41/21 - 3/14*x4 - 1/42*x5
x2 = -2/3 + 2/3*x5
x3 = -44/21 + 4/7*x4 - 1/21*x5
x4 is a free variable
x5 is a free variable


Infinite Solutions (3 equations, 5 variables with dependencies)

2x + 3y - z + w - 2v = 4

4x + 6y - 2z + 2w - 4v = 8

-2x - 3y + z - w + 2v = -4

In [76]:
# Test:
coefficients_array = np.array([
    [2, 3, -1, 1, -2],
    [4, 6, -2, 2, -4],
    [-2, -3, 1, -1, 2]
])

constants_array = np.array([4, 8, -4])

solution, status = solve_linear_equations(coefficients_array, constants_array)
print("Status:", status)
print("Solution:", solution)
# Format the parametric solution
formatted_solution = format_parametric_solution(str(solution))

# Print the formatted solution
for line in formatted_solution:
    print(line)


Status: Infinite solutions
Solution: 2.0 - 1.5*x2 - -0.5*x3 - 0.5*x4 - -1.0*x5, x4 is a free variable, x5 is a free variable
x1 = 2 - 3/2*x2 + 1/2*x3 - 1/2*x4 + 1*x5
x2 = x4 is a free variable
x3 = x5 is a free variable


m > n System (2 equations, 5 variables)

x + y + z + w + v = 10

2x + 3y + 2z + w + v = 15

In [77]:
# Test:
coefficients_array = np.array([
    [1, 1, 1, 1, 1],
    [2, 3, 2, 1, 1]
])

constants_array = np.array([10, 15])

solution, status = solve_linear_equations(coefficients_array, constants_array)
print("Status:", status)
print("Solution:", solution)
# Format the parametric solution
formatted_solution = format_parametric_solution(str(solution))

# Print the formatted solution
for line in formatted_solution:
    print(line)


Status: Infinite solutions
Solution: 15.0 - 1.0*x3 - 2.0*x4 - 2.0*x5, -5.0 - -1.0*x4 - -1.0*x5, x3 is a free variable, x4 is a free variable, x5 is a free variable
x1 = 15 - 1*x3 - 2*x4 - 2*x5
x2 = -5 + 1*x4 + 1*x5
x3 is a free variable
x4 is a free variable
x5 is a free variable


m < n (More equations than variables, no solutions)

x + y + z = 6

x + y - z = 4

2x - 3y + 2z = 1

3x + 2y + 3z = 12

x - y + z = 0

In [78]:
# Test:
coefficients_array = np.array([
    [1, 1, 1],
    [1, 1, -1],
    [2, -3, 2],
    [3, 2, 3],
    [1, -1, 1]
])

constants_array = np.array([6, 4, 1, 12, 0])

solution, status = solve_linear_equations(coefficients_array, constants_array)
print("Status:", status)
print("Solution:", solution)
# Format the parametric solution
formatted_solution = format_parametric_solution(str(solution))

# Print the formatted solution
for line in formatted_solution:
    print(line)


Status: No solution
Solution: None
x1 = None


Mixed Coefficients with Zero and Negative Values

-x + 2y - z + 0w + 3v = 0

2x - 4y + 0z - w - v = -2

0x + 3y + z + w - 5v = 3

In [88]:
# Test:
coefficients_array = np.array([
    [-1, 2, -1, 0, 3],
    [2, -4, 0, -1, -1],
    [0, 3, 1, 1, -5]
])

constants_array = np.array([0, -2, 3])

solution, status = solve_linear_equations(coefficients_array, constants_array)
print("Status:", status)
print("Solution:", solution)
# Format the parametric solution
formatted_solution = format_parametric_solution(str(solution))

# Print the formatted solution
for line in formatted_solution:
    print(line)


Status: Infinite solutions
Solution: 0.3333333333333333 - -0.16666666666666666*x4 - -2.1666666666666665*x5, 0.6666666666666666 - 0.16666666666666666*x4 - -0.8333333333333334*x5, 1.0 - 0.5*x4 - -2.5*x5, x4 is a free variable, x5 is a free variable
x1 = 1/3 + 1/6*x4 + 13/6*x5
x2 = 2/3 - 1/6*x4 + 5/6*x5
x3 = 1 - 1/2*x4 + 5/2*x5
x4 is a free variable
x5 is a free variable


>>Edge Cases

1x1 System - Unique Solution

5x = 25

In [89]:
# Test:
coefficients_array = np.array([
    [5]
])

constants_array = np.array([25])

solution, status = solve_linear_equations(coefficients_array, constants_array)
print("Status:", status)
print("Solution:", solution)
# Format the parametric solution
formatted_solution = format_parametric_solution(str(solution))

# Print the formatted solution
for line in formatted_solution:
    print(line)


Status: Unique solution
Solution: [5.]
x1 = [5.]


5x5 System - No Solution (inconsistent system)

x + y + z + w + v = 1

x + y + z + w + v = 2

x + y + z + w + v = 3

x + y + z + w + v = 4

x + y + z + w + v = 5

In [91]:
# Test:
coefficients_array = np.array([
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1]
])

constants_array = np.array([1, 2, 3, 4, 5])

solution, status = solve_linear_equations(coefficients_array, constants_array)
print("Status:", status)
print("Solution:", solution)
# Format the parametric solution
formatted_solution = format_parametric_solution(str(solution))

# Print the formatted solution
for line in formatted_solution:
    print(line)


Status: No solution
Solution: None
x1 = None
